In [25]:
import os
import json


# Load data/inputs/places.json
with open("../data/inputs/raw.json") as f:
    raw = json.load(f)
print(raw)

places = []
reviews = []
for row in raw:
    review_list = row.pop("reviews")
    for review in review_list:
        new_review = {}
        new_review["place_id"] = review["name"].split("/")[1]
        new_review["review_id"] = review["name"].split("/")[3]
        new_review["text"] = review["text"]["text"]
        new_review["rating"] = review["rating"]
        reviews.append(new_review)

    places.append(row)

print(places)
print(reviews)

# Save data/inputs/places.json
with open("../data/inputs/places.json", "w") as f:
    json.dump(places, f, ensure_ascii=False, indent=2)
with open("../data/inputs/reviews.json", "w") as f:
    json.dump(reviews, f, ensure_ascii=False, indent=2)

[{'id': 'ChIJoTcat9SMGGAR6GGG8zdcZvE', 'location': {'latitude': 35.6894807, 'longitude': 139.6916863}, 'rating': 4.4, 'userRatingCount': 5052, 'displayName': {'text': '東京都庁', 'languageCode': 'ja'}, 'reviews': [{'name': 'places/ChIJoTcat9SMGGAR6GGG8zdcZvE/reviews/ChZDSUhNMG9nS0VJQ0FnSUM5Ni1xUEFnEAE', 'relativePublishTimeDescription': '3 か月前', 'rating': 5, 'text': {'text': 'とにかく景色がよく無料で見れるのでいい観光地だと思います。人気観光地です。デートなど向いていると思います。\n観光案内所があるので次行くところも決められると思います。\n新宿駅からだと地下道を通ると5分くらいで行けます。動く歩道があります。\n有名人がよくピアノを弾いている都庁ピアノ展望台にがあります。\n富士山、筑波山、赤城山、東京タワー、スカイツリー、霞ヶ浦、東京ベイエリア、太平洋、東京湾などが見えます。\n中でも富士山はとにかく綺麗です。\n\nまた、\nインバウンドの影響で外国人が沢山いました。', 'languageCode': 'ja'}, 'originalText': {'text': 'とにかく景色がよく無料で見れるのでいい観光地だと思います。人気観光地です。デートなど向いていると思います。\n観光案内所があるので次行くところも決められると思います。\n新宿駅からだと地下道を通ると5分くらいで行けます。動く歩道があります。\n有名人がよくピアノを弾いている都庁ピアノ展望台にがあります。\n富士山、筑波山、赤城山、東京タワー、スカイツリー、霞ヶ浦、東京ベイエリア、太平洋、東京湾などが見えます。\n中でも富士山はとにかく綺麗です。\n\nまた、\nインバウンドの影響で外国人が沢山いました。', 'languageCode': 'ja'}, 'authorAttribution': {'displayName': '地

In [3]:
import os
import re
import time
import json
from tqdm import tqdm
from dotenv import load_dotenv
from openai import OpenAI


# Load the .env file
load_dotenv("../.env")
api_key = os.getenv("OPENAI_API_KEY")

client = OpenAI()

with open("../data/prompts/review_extension.txt") as f:
    review_extension = f.read()

with open("../data/inputs/reviews.json") as f:
    reviews = json.load(f)


features = []
for review in tqdm(reviews):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": review_extension},
            {"role": "user", "content": review["text"] + "\n\n[output]"},
        ],
    )
    sentences = response.choices[0].message.content
    sentences = [
        sentence.replace("- ", "").strip()
        for sentence in sentences.split("\n")
        if sentence != ""
    ]
    # re.sub(r"\d+\.", "", feature["feature"])
    sentences = [
        re.sub(r"\d+\.", "", sentence).strip()
        for sentence in sentences
        if sentence != ""
    ]
    for sentence in sentences:
        feature = {
            "place_id": review["place_id"],
            "review_id": review["review_id"],
            "feature": sentence,
        }
        features.append(feature)

    time.sleep(1)


with open("../data/inputs/features.json", "w") as f:
    json.dump(features, f, ensure_ascii=False, indent=2)

100%|██████████| 300/300 [35:33<00:00,  7.11s/it]


In [11]:
#import json
#import re

#with open("../data/inputs/features.json") as f:
#    features = json.load(f)

#new_features = []
#for feature in features:
#    # Replace 1. 2. 3. with ""
#    feature["feature"] = re.sub(r"\d+\.", "", feature["feature"]).strip()
#    new_features.append(feature)

#with open("../data/inputs/features.json", "w") as f:
#    json.dump(new_features, f, ensure_ascii=False, indent=2)


In [12]:
import os
import time
import json
from tqdm import tqdm
from dotenv import load_dotenv
from openai import OpenAI


# Load the .env file
load_dotenv("../.env")
api_key = os.getenv("OPENAI_API_KEY")

client = OpenAI()

with open("../data/inputs/features.json") as f:
    features = json.load(f)

embeddings = []
for feature in tqdm(features):
    response = client.embeddings.create(
        input=feature["feature"],
		model="text-embedding-3-small"
    )
    feature["embedding"] = response.data[0].embedding
    embeddings.append(feature)

    time.sleep(1)


with open("../data/inputs/embeddings.json", "w") as f:
    json.dump(embeddings, f, ensure_ascii=False, indent=2)

  0%|          | 7/4093 [00:08<1:26:20,  1.27s/it]


KeyboardInterrupt: 

In [1]:
import json

# Load features.json and add key to number
with open("../data/inputs/features.json") as f:
	features = json.load(f)

new_features = []
for i, feature in enumerate(features):
	feature["key"] = i
	new_features.append(feature)

# Save features.json
with open("../data/inputs/features.json", "w") as f:
	json.dump(new_features, f, ensure_ascii=False, indent=2)